#### Task1 

In [59]:
from math import exp

In [26]:
INF = 10
EPS = 1e-8
ITERATIONS_NUM = 1000

In [65]:
class Differentiable:
    def __init__(self, derivatives):
        self.derivatives = derivatives
    def __call__(self, x):
        return self.derivatives[0](x)
    def grad(self):
        if (len(self.derivatives) == 1):
            raise Exception("no derivatives were provided")
        return Differentiable(self.derivatives[1:])

class Polynom:
    def __init__(self, coefs):
        self.coefs = coefs
        self._degree = len(coefs) - 1
    def __call__(self, x):
        res = 0
        for i, coef in enumerate(self.coefs):
            res += (x ** i) * coef
        return res
    def get_degree(self):
        return self._degree
    def grad(self):
        grad_coefs = [0] * self._degree
        for i in range(1, self._degree + 1):
            grad_coefs[i - 1] = self.coefs[i] * i
        return Polynom(grad_coefs)

In [43]:
def bisec(p, l, r):
    assert p(r) * p(l) < 0
    sign = 1 if p(r) > 0 else -1

    while (r - l > EPS):
        m = (r + l) / 2
        if (p(m) * sign > 0):
            r = m
        else:
            l = m
    return l

def newton(p):
    x = 1
    p_grad = p.grad()
    
    for i in range(ITERATIONS_NUM):
        x = x - p(x) / p_grad(x)
    return x

In [44]:
def get_polynom(k, a):
    coefs = [0] * (k + 1)
    coefs[k] = 1
    coefs[0] = -a
    return Polynom(coefs)

In [45]:
p = get_polynom(2, 2)

In [46]:
print(f'bisec: {bisec(p, 0, INF)}')
print(f'newton: {newton(p)}')

bisec: 1.414213553071022
newton: 1.414213562373095


### Task2

In [53]:
def get_roots(p):
    if (p.get_degree() == 1):
        return [-p.coefs[0] / p.coefs[1]]

    a = [-INF]
    a += get_roots(p.grad())
    a.append(INF)
    
    roots = []
    for i in range(len(a) - 1):
        roots.append(bisec(p, a[i], a[i + 1]))
    return roots

In [49]:
def get_polynom_by_roots(roots):
    coefs = [0] * (len(roots) + 1)
    for mask in range(1 << len(roots)):
        product = 1
        bits = 0
        for i in range(len(roots)):
            if ((mask >> i) & 1):
                product *= - roots[i]
                bits += 1
        coefs[len(roots) - bits] += product
    return Polynom(coefs)

In [55]:
get_roots(get_polynom_by_roots([1, 2, 3, 4, 5]))

[0.9999999987523025,
 1.999999993185262,
 2.999999996626574,
 3.999999993366687,
 4.999999993699136]

Понятно, что этот код не работает в самой общей постановке задачи. Как минимум он всегда возвращает столько корней, какой степени полином. Он совершенно не работает в сценарии, когда у нас есть кратные корни. Да и у производной могут быть кратные корни, но мне очень не хотелось разбирать все эти случаи. Так что вот так.

### Task03

In [68]:
def get_differentiable(a, b, c, d):
    def f(x):
        return exp(a * x) + exp(- b * x) + c * ((x - d) ** 2)
    
    def f_grad(x):
        return a * exp(a * x) - b * exp(- b * x) + 2 * c * (x - d)
    
    def f_grad2(x):
        return (a ** 2) * exp(a * x) + (b ** 2) * exp(- b * x) + 2 * c 
    
    return Differentiable([f, f_grad, f_grad2])

In [69]:
f = get_differentiable(1, 1, 1, 1)

In [ ]:
def ternary_search(f):
    l = -INF
    r = INF
    
    while(r - l > EPS):
        m1 = l + ((r - l) / 3)
        m2 = l + (2 * (r - l) / 3)
        if (f(m1) > f(m2)):
            l = m1
        

In [72]:
print("bisec: ", bisec(f.grad(), -INF, INF))
print("newton: ", newton(f.grad()))

bisec:  0.49007306806743145
newton:  0.4900730684805478
